In [21]:
#we are looking at the group algebra F_p[C_N]
#this is F_p[x]/(x^N-1)
#if p|N, we can write (x^N-1) = (x^m-1)^{p^s} for some s
#we are looking at roots of unity mod p
#to factor x^m-1, use cyclotomic polynomials: x^m-1 = \prod_{d|m} phi_d(x)
#to factor phi_d(x) in F_p, that's equivalent to looking at (p) in Z[x]/Phi_d(x) = Z[zeta]
#(p) = P_1...P_g where each P_i has the same residue degree f
#Here f*g = phi(d), and f is the order of p modulo d, phi is Euler totient
#phi_d(x) factors in F_p[x] into g polynomials, each of degree f
#thus F_p[x]/(x^N-1) \cong \prod_{d|m} \prod_{i=1}^g F_p[x]/(P_i^{p^s})
#by the Chinese remainder theorem
#i.e. just factor x^N-1 mod p, and map onto residue classes

In [9]:
#h is an element of F_p[C_N]
#that is, h = h0+h1*x+h2x^2+...+h_{N-1}x^N-1
#we allow h as a list of N numbers modulo p
#h = [h0,h1,h2,...,h_{N-1}]
def discrete_fourier_transform(h,N,p):
    #define the polynomial ring F_p[x]
    R = PolynomialRing(GF(p),'x')
    #name the generator x an element of R
    x = R.0
    #define the quotient ring F_p[x]/(x^N-1)
    S = R.quotient(x^N - 1, 'z')
    #define the polynomial x^N-1
    f = x**N-1; assert f in R
    #transform the list of coefficients of h into a polynomial in R=F_p[x]
    h = sum(h[i]*x**i for i in range(N)); assert h in S
    #factor f in F_p[x], save as list of factors and multiplicities
    f_factors = list(f.factor())
    #implement the Chinese remainder theorem mapping S=F_p[x]/(x^N-1) --> \prod_i R/(factor_i^mult_i)
    h_transform=[list(R.quotient(f_factors[i][0]**f_factors[i][1])(h)) for i in range(len(f_factors))]
    return h_transform

In [23]:
discrete_fourier_transform([0,1,0,5,0,0,6,1,0,1],10,17)

[[15], [14], [16, 6, 0, 4], [1, 11, 0, 4]]

In [14]:
R = PolynomialRing(GF(7^4),'x')

In [15]:
x = R.0

In [16]:
N=10

In [17]:
f = x**N-1; assert f in R

In [18]:
f.factor()

(x + 1) * (x + 6) * (x + 3*z4^2 + 5*z4 + 6) * (x + 4*z4^2 + 2*z4 + 1) * (x + z4^3 + 3*z4 + 5) * (x + z4^3 + 6*z4^2 + 2*z4 + 6) * (x + 2*z4^3 + 2*z4^2 + 3*z4 + 2) * (x + 5*z4^3 + 5*z4^2 + 4*z4 + 5) * (x + 6*z4^3 + 4*z4 + 2) * (x + 6*z4^3 + z4^2 + 5*z4 + 1)

In [23]:
z4=GF(7^4).gens()[0]

In [37]:
(3*z4^2 + 5*z4 + 6)^5

6